# 데이터 전처리

In [47]:
import pandas as pd
import re
import hanja
from hanja import hangul
from collections import Counter
import nltk
from kiwipiepy import Kiwi

In [2]:
data = pd.read_csv("/data/ephemeral/home/level2-nlp-datacentric-nlp-10/train_denoised_none.csv")

## 1. None 데이터 제거

In [3]:
# 'text' 열에서 None 값이 포함된 행 제거
df_cleaned = data[data['text'].notna()]
df_cleaned

,ID,text,target
0,ynat-v1_train_00000,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,4
1,ynat-v1_train_00001,검찰청에 로또 당첨자 회장에 2억 달러 송금,3
2,ynat-v1_train_00002,北 김정은 자주통일 새 시대 열어나가야 보자,2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
2797,ynat-v1_train_02797,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


## 2. 특수기호 처리

### 2-1. 특수 기호 제거

In [5]:
def remove_special_char(text):
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [6]:
df_cleaned['cleaned_text_1'] = df_cleaned['text'].apply(remove_special_char)
df_cleaned

/tmp/ipykernel_235829/2476612250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['cleaned_text_1'] = df_cleaned['text'].apply(remove_special_char)


,ID,text,target,cleaned_text_1
0,ynat-v1_train_00000,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,4,정영석 파파미사일 KT에 이용기간 2년 단축 2분기종료2보
1,ynat-v1_train_00001,검찰청에 로또 당첨자 회장에 2억 달러 송금,3,검찰청에 로또 당첨자 회장에 2억 달러 송금
2,ynat-v1_train_00002,北 김정은 자주통일 새 시대 열어나가야 보자,2,김정은 자주통일 새 시대 열어나가야 보자
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,6,대선 앞두고 중국인 단체 발언서 비공개로 감시 강화
...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠
2797,ynat-v1_train_02797,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,4,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정


### 2-2. 특수 기호 처리

In [16]:
# 특수기호와 변환할 문자열 매핑
replacement_dict = {
    'U+': '유플러스',
    '%': '퍼센트',
    '…': ' ',
    '.': '-',
    '~': '-',
    '..': '-',
    '"': '-',
    "'": '-',
    '·': '-',
    '↔': '-',
    '→': '에서',
    '↑': '상승',
    'ｍ': '미터',
    '㎝': '센티미터',
    '㎜': '밀리미터'
}

In [17]:
# 텍스트 열의 특수기호를 교체하는 함수
def replace_special_chars(text):
    for char, replacement in replacement_dict.items():
        text = text.replace(char, replacement)
    return text

In [18]:
# 데이터프레임의 'text' 열에 함수 적용
df_cleaned['cleaned_text_2'] = df_cleaned['text'].apply(replace_special_chars)

/tmp/ipykernel_235829/2115331316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['cleaned_text_2'] = df_cleaned['text'].apply(replace_special_chars)


In [15]:
df_cleaned

,ID,text,target,cleaned_text_1,cleaned_text_2
0,ynat-v1_train_00000,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,4,정영석 파파미사일 KT에 이용기간 2년 단축 2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축-2분기종료2보
1,ynat-v1_train_00001,검찰청에 로또 당첨자 회장에 2억 달러 송금,3,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금
2,ynat-v1_train_00002,北 김정은 자주통일 새 시대 열어나가야 보자,2,김정은 자주통일 새 시대 열어나가야 보자,北 김정은 자주통일 새 시대 열어나가야 보자
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통-시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,6,대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화
...,...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨-10점만점에 12점도
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠
2797,ynat-v1_train_02797,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,4,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운-서비스 정상화종료2보
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해-승부는 냉정


## 3. 한자 처리

### 3-1. 한자 제거

In [19]:
def remove_hanja(text):
    text = re.sub(r'[\u4E00-\u9FFF]', ' ', text)  # 한자 제거
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [20]:
df_cleaned['cleaned_text_3'] = df_cleaned['text'].apply(remove_hanja)
df_cleaned

/tmp/ipykernel_235829/1063134254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['cleaned_text_3'] = df_cleaned['text'].apply(remove_hanja)


,ID,text,target,cleaned_text_1,cleaned_text_2,cleaned_text_3
0,ynat-v1_train_00000,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,4,정영석 파파미사일 KT에 이용기간 2년 단축 2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축 2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보
1,ynat-v1_train_00001,검찰청에 로또 당첨자 회장에 2억 달러 송금,3,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금
2,ynat-v1_train_00002,北 김정은 자주통일 새 시대 열어나가야 보자,2,김정은 자주통일 새 시대 열어나가야 보자,北 김정은 자주통일 새 시대 열어나가야 보자,김정은 자주통일 새 시대 열어나가야 보자
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,6,대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,대선 앞두고 중국인 단체 발언서 비공개로 감시 강화
...,...,...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠
2797,ynat-v1_train_02797,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,4,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정


### 3-2. 한자 -> 한글로 변환

In [23]:
def convert_hanja_to_korean(text):
    text = hanja.translate(text, 'substitution')
    return text

In [24]:
df_cleaned['cleaned_text_4'] = df_cleaned['text'].apply(convert_hanja_to_korean)
df_cleaned

/tmp/ipykernel_235829/522468202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['cleaned_text_4'] = df_cleaned['text'].apply(convert_hanja_to_korean)


,ID,text,target,cleaned_text_1,cleaned_text_2,cleaned_text_3,cleaned_text_4
0,ynat-v1_train_00000,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,4,정영석 파파미사일 KT에 이용기간 2년 단축 2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축 2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보
1,ynat-v1_train_00001,검찰청에 로또 당첨자 회장에 2억 달러 송금,3,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금
2,ynat-v1_train_00002,北 김정은 자주통일 새 시대 열어나가야 보자,2,김정은 자주통일 새 시대 열어나가야 보자,北 김정은 자주통일 새 시대 열어나가야 보자,김정은 자주통일 새 시대 열어나가야 보자,북 김정은 자주통일 새 시대 열어나가야 보자
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,6,대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,미대선 앞두고 중국인 단체 발언서 비공개로 감시 강화
...,...,...,...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠
2797,ynat-v1_train_02797,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,4,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정


## 4. 불용어 제거

- 단어의 빈도수를 분석하여 모든 target에서 불필요하게 많이 등장한 단어 제거

In [33]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [48]:
kiwi = Kiwi()

In [49]:
# 형태소 분석 후 단어 추출 함수
def extract_nouns(text):
    tokens = kiwi.analyze(text)[0][0]  # 첫 번째 결과의 첫 번째 토큰 리스트 사용
    nouns = [word.form for word in tokens if word.tag.startswith('N')]  # 명사만 추출
    return nouns

In [50]:
# 단어 빈도수 계산 함수
def get_word_counts_by_category(df, text_col, target_col):
    category_word_counts = {}

    for category in df[target_col].unique():
        category_text = df[df[target_col] == category][text_col].str.cat(sep=' ')
        tokens = sum([extract_nouns(sentence) for sentence in category_text.split()], [])
        category_word_counts[category] = Counter(tokens)

    return category_word_counts

In [29]:
# 단어 빈도수 계산 함수
def get_word_counts_by_category(df, text_col, target_col):
    category_word_counts = {}
    
    for category in df[target_col].unique():
        category_text = df[df[target_col] == category][text_col].str.cat(sep=' ')
        tokens = nltk.word_tokenize(category_text)
        category_word_counts[category] = Counter(tokens)
    
    return category_word_counts


In [51]:
# 모든 카테고리의 단어 빈도수 분석
word_counts_by_category = get_word_counts_by_category(df_cleaned, 'text', 'target')

In [52]:
# 모든 카테고리에서 공통으로 많이 등장한 단어 찾기
total_word_counts = Counter()
for category, word_counts in word_counts_by_category.items():
    total_word_counts += word_counts

In [56]:
# 불필요하게 많이 등장한 단어 임계치 설정
threshold = 50  # 이 값은 조정 가능

In [57]:
# 임계치 이상의 빈도로 등장하는 단어를 불용어로 지정
common_words = [word for word, count in total_word_counts.items() if count > threshold]

In [58]:
common_words

['년',
 '위',
 '만',
 '대',
 '한국',
 '서',
 '종합',
 '월',
 '일',
 '대통령',
 '천',
 '억',
 '원',
 '등',
 '팀',
 '명',
 '이란',
 '대표',
 '프로',
 '공감']

In [38]:
# 불용어 제거 함수
def remove_common_words(text, common_words):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in common_words]
    return ' '.join(filtered_tokens)

In [39]:
df_cleaned['cleaned_text_5'] = df_cleaned['text'].apply(lambda x: remove_common_words(x, common_words))
df_cleaned

/tmp/ipykernel_235829/1025694711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['cleaned_text_5'] = df_cleaned['text'].apply(lambda x: remove_common_words(x, common_words))


,ID,text,target,cleaned_text_1,cleaned_text_2,cleaned_text_3,cleaned_text_4,cleaned_text_5
0,ynat-v1_train_00000,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,4,정영석 파파미사일 KT에 이용기간 2년 단축 2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축 2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보,정영석 파파미사일 KT에 이용기간 2년 단축…2분기종료2보
1,ynat-v1_train_00001,검찰청에 로또 당첨자 회장에 2억 달러 송금,3,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금,검찰청에 로또 당첨자 회장에 2억 달러 송금
2,ynat-v1_train_00002,北 김정은 자주통일 새 시대 열어나가야 보자,2,김정은 자주통일 새 시대 열어나가야 보자,北 김정은 자주통일 새 시대 열어나가야 보자,김정은 자주통일 새 시대 열어나가야 보자,북 김정은 자주통일 새 시대 열어나가야 보자,자주통일 시대 열어나가야 보자
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,6,대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,미대선 앞두고 중국인 단체 발언서 비공개로 감시 강화,美대선 앞두고 중국인 단체 발언서 비공개로 감시 강화
...,...,...,...,...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,갤럭시S9 정식 주말 이통시장 잠잠
2797,ynat-v1_train_02797,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,4,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,텔레그램 한인 아시아서 2시간 다운…서비스 정상화종료2보
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,인터뷰 친구에게 안타 맞는 싫어해…승부는 냉정


## 5. 카테고리별 Keyword 단어 추가

- 카테고리별 빈도 10 이상 단어들을 리스트로 추출 후, 해당 카테고리에만 존재하는 고유 키워드 vocab 생성

- vocab을 각 카테고리의 텍스트에 이어 붙여 모델이 강한 학습을 가지고 학습

In [59]:
# 형태소 분석 후 단어 추출 함수
def extract_nouns(text):
    tokens = kiwi.analyze(text)[0][0]  # 첫 번째 결과의 첫 번째 토큰 리스트 사용
    nouns = [word.form for word in tokens if word.tag.startswith('N')]  # 명사만 추출
    return nouns

In [60]:
# 단어 빈도수 계산 함수
def get_word_counts_by_category(df, text_col, target_col):
    category_word_counts = {}

    for category in df[target_col].unique():
        category_text = df[df[target_col] == category][text_col].str.cat(sep=' ')
        tokens = sum([extract_nouns(sentence) for sentence in category_text.split()], [])
        category_word_counts[category] = Counter(tokens)

    return category_word_counts

In [61]:
# 모든 카테고리의 단어 빈도수 분석
word_counts_by_category = get_word_counts_by_category(df_cleaned, 'text', 'target')

In [62]:
# 상위 50개 단어 추출
category_vocab = {}
for category, word_counts in word_counts_by_category.items():
    # 빈도가 높은 상위 50개 단어만 추출
    sorted_vocab = [word for word, _ in word_counts.most_common(50)]
    category_vocab[category] = set(sorted_vocab)

In [63]:
# 각 카테고리에서 고유한 단어 추출
unique_vocab = {}
for category, vocab in category_vocab.items():
    other_categories_vocab = set().union(*(category_vocab[cat] for cat in category_vocab if cat != category))
    unique_words = vocab - other_categories_vocab
    unique_vocab[category] = list(unique_words)

In [64]:
# 결과 출력
for category, words in unique_vocab.items():
    print(f"Category {category} unique vocab: {words}")

Category 4 unique vocab: ['아이폰', '앱', '전자', '시작', '통신', '애플', '폰', '스마트폰', '인공', '지능', '구글', '갤럭시', '국내']
Category 3 unique vocab: ['대구', '주년', '경찰', '학교', '충북', '코로나', '국제', '경남', '방송', '과학', '교육청', '기자', '교육', '공무원', '추진', '대전']
Category 2 unique vocab: ['국민', '정치', '위원회', '북', '국정', '경제', '남북', '국회', '김정', '당', '예비', '평화', '정', '후보', '중단', '언론', '민주', '길']
Category 5 unique vocab: ['판매', '상승', '실적', '금리', '작년', '금융', '코스피', '주택', '시간', '배', '주가', '하락', '은행', '분기', '주식', '증권', '주', '영업익', '대출', '내년', '조', '펀드', '올해']
Category 6 unique vocab: ['중국', '반대', '이스라엘', '사우디', '공격', '미사일', '아프리카', '제재', '이탈리아', '최고', '인', '사망', '프랑스', '조사', '방문', '여성', '시설', '시위', '합의']
Category 0 unique vocab: ['뉴스', '개최', '예술', '경북', '전국', '먼지', '도', '날씨', '낮', '축제', '기', '비', '사진', '말', '미세', '공연', '신간', '일자리']
Category 1 unique vocab: ['강', '감독', '아시안', '승리', '야구', '골', '베트남', '연속', '시즌', '선수', '점', '아스', '계약', '롯데', '컵', '날', '연', '월드컵', '축구', '연패', '승', '배구', '공감', '리그', '연승', '우승']
